
# 红外传感器与摄像头

## 1、红外距离传感器


红外测距传感器具有一对红外信号发射与接收二极管，利用的红外测距传感器发射出一束红外光，在照射到物体后形成一个反射的过程，反射到传感器后接收信号，然后利用CCD图像处理接收发射与接收的时间差的数据。经信号处理器处理后计算出物体的距离。


小车上使用的红外距离传感器相当于一个电子开关，检测到障碍输出低电平，平时为高电平，可以通过调节电位器改变检测距离


## 2、红外巡线传感器


红外巡线传感器 有一个红外发射管和一个红外接收管，当地面为黑色 吸收所有光， 所以接受管不会导通当地面为白色，反射所有光，所以接收管导通，其工作原理与距离传感器基本相同


### 测试程序

In [ ]:

import RPi.GPIO as GPIO
import time


导入GPIO库后定义小车的所有红外传感器接口，并且对接口进行初始化

In [ ]:

class CarInfrared(object):
    def __init__(self):
        self.GPIO_Infrared_right = 8
        self.GPIO_Infrared_left = 7

        self.GPIO_left_tracking = 16
        self.GPIO_right_tracking = 12

        GPIO.setup(self.GPIO_Infrared_right, GPIO.IN)
        GPIO.setup(self.GPIO_Infrared_left, GPIO.IN)

        GPIO.setup(self.GPIO_left_tracking, GPIO.IN)
        GPIO.setup(self.GPIO_right_tracking, GPIO.IN)

    def InfraredMeasure(self):
        left_measure = GPIO.input(self.GPIO_Infrared_left)
        right_measure = GPIO.input(self.GPIO_Infrared_right)

        return [left_measure, right_measure]

    def TrackingMreasure(self):
        left_tracking = GPIO.input(self.GPIO_left_tracking)
        right_tracking = GPIO.input(self.GPIO_right_tracking)#1黑0白

        return [left_tracking, right_tracking]


测试传感器输出

In [ ]:

try:
    car = CarInfrared()
    while True:
        [left, right] = car.InfraredMeasure()
        [l, r] = car.TrackingMreasure()
        # print(left, right)
        print(l, r)
        time.sleep(1)

    # Reset by pressing CTRL + C
except KeyboardInterrupt:
    print("Measurement stopped by User")
    GPIO.cleanup()


## 3、摄像头


树莓派使用CSI摄像头，其主要特征：
全称CMOS Sensor Interface(CSI)<br/>
前端-配置逻辑接口以便支持大多数通用的可用的CMOS摄像头接口。<br/>
支持CCIR656视频接口以及传统的摄像头接口。<br/>
8位数据口，支持方便YCC,YUV,Bayer或者是RGB的数据格式输入。<br/>
完全可控的8-bit或16-bit数据到32-bit的FIFO进行打包<br/>
32*32大小的FIFO存储接受到的的图像像素数据，该FIFO可以通过可编程的IO或者是DMA进行读取.<br/>
后端-提供了直接到eMMA的预处理PrP块接口(PrP和PP组成了MX27图形加速器eMMA，PrP和PP能够用来给视频做预处理和后期处理，例如，放大，缩小，颜色转换)<br/>
提供sensor的可屏蔽中断源，该中断源也是中断可控的： 开始Frame,结束Frame,Change of Field,FIFO Full<br/>
提供给外部sensor用的，可配置的主时钟频率<br/>
由统计数据产生的自动曝光(AE)和自动白平衡(AWB)控制。<br/>

感觉不是什么重点，参考链接：[CMOS Sensor Interface(CSI)](https://www.cnblogs.com/lifan3a/articles/5010633.html)


可以使用socket获取树莓派摄像头图像，通过网络传输，使用opencv库在PC端对接受的数据进行解码，此时的PC段与树莓派可以看做一个C/S结构的图像传输应用


### 示例代码（PC端）

In [1]:

import cv2
import numpy
import socket
import struct


通过socket套接字接受数据，然后使用opencv对收到的二进制数据进行解码，并显示在窗口中。

In [ ]:

HOST='192.168.12.183'
PORT= 8000
buffSize=65535


server=socket.socket(socket.AF_INET,socket.SOCK_DGRAM) #创建socket对象
server.bind((HOST,PORT))
print('now waiting for frames...')
i = 7

while True:
    data,address=server.recvfrom(buffSize) #先接收的是字节长度
    if len(data)==1 and data[0]==1: #如果收到关闭消息则停止程序
        server.close()
        cv2.destroyAllWindows()
        exit()
    if len(data)!=4: #进行简单的校验，长度值是int类型，占四个字节
        length=0
        continue
    else:
        length=struct.unpack('i',data)[0] #长度值
    data,address=server.recvfrom(buffSize) #接收编码图像数据
    if length!=len(data): #进行简单的校验
        print("check error")
        continue
    data=numpy.array(bytearray(data)) #格式转换
    imgdecode=cv2.imdecode(data,1) #解码
    print('have received frame')

    cv2.imshow('frames',imgdecode) #窗口显示

    key = cv2.waitKey(1)
    if key == ord('s'):
        cv2.imwrite('/home/mingrui/Mingrui/RaspberryCar/photo_tennis/tennis_'+ str(i) +'.jpg', imgdecode)
        i = i + 1


    if cv2.waitKey(1)==27: #按下“ESC”退出
        break
    
server.close()
cv2.destroyAllWindows()

now waiting for frames...
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
check error
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
have received frame
ha


上述代码正常执行需要在树莓派的终端上首先执行，并且脚本中的IP地址应为当前PC的IP地址 <br/><br/>
sudo python3 ./PythonCode/camera.py